# Bayesian Logistic Uncertain Reaction (BLUR) Model
* Bayesian statistical approach to calibrating thermodynamic databases
* **Past calibration methods all assume perfect thermodynamic equilibrium**
* Equilibrium assumption produces step-function reaction boundaries, which look nothing like real experimental data
* **BLUR model smears out reaction boundaries** resulting from both:
    * ***measurement uncertainties***
    * ***reaction kinetics***
* Bayesian approach enables straightforward inclusion of prior constraints
* Built entirely with robust probability models **resistant to outlier bias**

## Import libraries ...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', None)
from scipy import stats

import sys
sys.path.insert(0, '..')
sys.path.insert(0, '/Users/msghiorso/ENKI/PhaseObjC')
from thermoengine import calibrate
import thermoengine

import importlib
importlib.reload(thermoengine)
importlib.reload(calibrate)

## Measurement Uncertainties soften reaction boundaries
### Thermodynamic Equilibrium
* $\Delta G_{R} < 0$: Forward Reaction
* $\Delta G_{R} > 0$: Reverse Reaction

In [ ]:
dGrxn_a = np.linspace(-10,10,1001)
sig=1
Ptrans_a = 1-calibrate.DatabaseModel.rxn_trans_fun(dGrxn_a/sig)
steptrans_a = dGrxn_a<0
plt.subplot(211)
plt.plot(dGrxn_a,steptrans_a,'k--',label='Ideal Rxn')
plt.plot(dGrxn_a,Ptrans_a,'k-',label='Uncertain Rxn')
plt.xlim(-3,3)
plt.ylabel('Probability')
plt.legend(loc='upper right')

plt.subplot(212)
plt.plot(dGrxn_a,stats.norm.pdf(dGrxn_a),'k-',label=r'$\Delta G_R$ error')
plt.errorbar(0,0,xerr=1,fmt='ko',capsize=4)
plt.xlabel(r'$\Delta G_R$')
plt.ylabel('Probability')
plt.legend(loc='upper right')
plt.xlim(-3,3)
plt.savefig('figs/test.eps')
plt.show()

### Rxn Energy Uncertainty
* Total uncertainty, $\sigma_{\Delta G}$, is determined by error propagation
* independent errors add in quadrature:
$$\sigma_{\Delta G}^2 = \left(\frac{\partial G}{\partial T} \cdot \sigma_T\right)^2 + \left( \frac{\partial G}{\partial P} \cdot \sigma_P \right)^2$$

$$\sigma_{\Delta G}^2 = \left(\Delta S_R \cdot \sigma_T\right)^2 + \left( \Delta V_R \cdot \sigma_P \right)^2$$

* For applications with solution phases, there are additional terms associated with compositional uncertainty of each phase

## Reaction kinetics separates forward & reverse reactions
* Observed reaction direction is made uncertain by kinetics
* Factors like time, temperature, and presence of water greatly affect whether reaction is observable within a given experiment
* kinetic factors combine to produce an energy threshold for each reaction $G_R^{\rm th}$
* the ideal reaction is likely observable only when $\left|\Delta G_R\right| > \left|G_R^{\rm th}\right|$

In [ ]:
dGrxn_a = np.linspace(-10,10,1001)
sigG=0.2
Gth = 1
logPfwd_a = calibrate.DatabaseModel.logprob_fwd(dGrxn_a,Gth,sigG)
logPrev_a = calibrate.DatabaseModel.logprob_rev(dGrxn_a,Gth,sigG)
logPnc_a = calibrate.DatabaseModel.logprob_nc(dGrxn_a,Gth,sigG)

plt.plot(dGrxn_a,np.exp(logPfwd_a),'r-',lw=2,label='FWD')
plt.plot(dGrxn_a,np.exp(logPrev_a),'b-',lw=2,label='REV')
plt.plot(dGrxn_a,np.exp(logPnc_a),'k-',lw=2,label='NC')

plt.xlim(-3,3)
plt.ylabel('Probability')
plt.xlabel(r'$\Delta G_R \; / \; G_R^{\rm th}$')
plt.legend(loc='lower right',title='Rxn. Dir.')
plt.show()


## Reaction Threshold Energy depends on experimental conditions
* Energy threshold $G_R^{\rm th}$ can vary dramatically in response to experimental conditions
* We introduce unitless reaction parameters ($\alpha_i$), to capture linear dependence on experimental conditions ($x_i$) :
$$\log \left( \frac{G_R^{\rm th}}{\rm J/mol} \right) = \sum_i \alpha_i x_i$$
* The **most important modeled parameters** are:
    * **time**: $x_t = \log\left(\frac{\rm time}{\rm hr}\right)$
    * **temperature**: $x_T = \frac{T}{\rm Kelvin}$
    * **water**: $x_w =$ 1 if water present (otherwise 0)
    * **reaction studied**: $x_R = 1$ for reaction $R$ (otherwise 0)
    
* For positive values ($\alpha_i > 0$), $x_i$ increases the threshold energy
* For negative values ($\alpha_i < 0$), $x_i$ decreases the threshold energy
* Values near zero indicate weak dependence on $x_i$


## Logistic Ensures Robust Calibration
* **Cost function grows much slower for logistic vs normal transtion function**
* Outliers only contribute moderately to overall fitness, **limiting calibration bias**


In [ ]:
logPfwd_norm_a = calibrate.DatabaseModel.logprob_fwd(dGrxn_a,Gth,sigG,rxn_trans_typ='normal')
logPrev_norm_a = calibrate.DatabaseModel.logprob_rev(dGrxn_a,Gth,sigG,rxn_trans_typ='normal')
logPnc_norm_a = calibrate.DatabaseModel.logprob_nc(dGrxn_a,Gth,sigG,rxn_trans_typ='normal')

plt.plot(dGrxn_a,-logPfwd_a,'r-',lw=2)
plt.plot(dGrxn_a,-logPrev_a,'b-',lw=2)
plt.plot(dGrxn_a,-logPnc_a,'k-',lw=2,label='Logistic')
plt.plot(dGrxn_a,-logPfwd_norm_a,'r:')
plt.plot(dGrxn_a,-logPrev_norm_a,'b:')
plt.plot(dGrxn_a,-logPnc_norm_a,'k:',label='Normal')

plt.ylim(-1,36)
plt.xlim(-3,3)
plt.ylabel(r'Cost = $-\log_e$ Prob')
plt.xlabel(r'$\Delta G_R \; / \; G_R^{\rm th}$')
plt.legend(loc='lower right')
plt.show()

## Define Plotting/Analysis Functions

In [ ]:
def calc_rxn_trans(dGrxn_a,sigG_a):
    Gth = 1.0
    rxn_trans_l = []
    for ind,sigG in enumerate(sigG_a):
        logprob_fwd_a = calibrate.DatabaseModel.logprob_fwd(dGrxn_a, Gth, sigG)
        logprob_rev_a = calibrate.DatabaseModel.logprob_rev(dGrxn_a, Gth, sigG)
        logprob_nc_a = calibrate.DatabaseModel.logprob_nc(dGrxn_a, Gth, sigG)
        rxn_trans_df = pd.DataFrame({'dGrxn_Gth':dGrxn_a/Gth,'Gth':Gth,'FWD':logprob_fwd_a,
                                     'REV':logprob_rev_a,'NC':logprob_nc_a})
        rxn_trans_l.append(rxn_trans_df)
    
    return rxn_trans_l

def plot_rxn_trans(rxn_trans_l, sigG_a):
    lines_fwd = []
    lines_rev = []
    lines_nc = []
    NsigG = len(rxn_trans_l)
    
    for ind,rxn_trans_df in enumerate(rxn_trans_l):
        frac = (NsigG-1.1*ind)/NsigG
        rcol = frac*np.array([1,0,0])+(1-frac)*np.array([1,1,1])
        bcol = frac*np.array([0,0,1])+(1-frac)*np.array([1,1,1])
        kcol = frac*np.array([0,0,0])+(1-frac)*np.array([1,1,1])
        
        h_fwd, = plt.plot(rxn_trans_df['dGrxn_Gth'], np.exp(rxn_trans_df['FWD']),'-',
                          color=rcol,lw=3)
        h_rev, = plt.plot(rxn_trans_df['dGrxn_Gth'], np.exp(rxn_trans_df['REV']),'-',
                          color=bcol,lw=3)
        h_nc, = plt.plot(rxn_trans_df['dGrxn_Gth'], np.exp(rxn_trans_df['NC']),'-',
                         color=kcol,lw=3)
        lines_fwd.append(h_fwd)
        lines_rev.append(h_rev)
        lines_nc.append(h_nc)
        
    # Label Plots
    plt.xlabel(r'$\Delta G_{rxn} \; / \; G^{th}_{rxn}$')
    plt.ylabel('Probability')
    plt.ylim(0,1.13)
    plt.xlim(-3,+3)
    
    # Label Curves
    plt.text(-2.5,1.03,'FWD',color='r',fontsize=14,horizontalalignment='center')
    plt.text(+2.5,1.03,'REV',color='b',fontsize=14,horizontalalignment='center')
    plt.text(0.0,1.03,'NC',color='k',fontsize=14,horizontalalignment='center')
    
    plt.legend(lines_nc,sigG_a,loc='right',
               title=r'$\sigma_{\Delta G}  \; / \; G^{th}_{rxn}$')
    
    plt.show()




In [ ]:
## Data Quality determined by both Measurement Uncertainty and Kinetics


In [ ]:
dGrxn_a = np.linspace(-3,3,1001)
sigG_a = np.array([0.1,0.4,0.8])
rxn_trans_l = calc_rxn_trans(dGrxn_a, sigG_a)
plot_rxn_trans(rxn_trans_l, sigG_a)
plt.savefig('rxn-transition-schematic.eps', bbox_inches='tight')


## Bayes Theorem used to perform calibration
* Want to infer **posterior: model family of acceptable parameters given the data**
* Bayes Theorem:
$$P({\rm model}\;|\;{\rm data}) \propto P({\rm data}\;|\;{\rm model}) \cdot P({\rm model})$$
$${\rm Posterior} \propto {\rm Likelihood} \cdot {\rm Prior}$$
* Thus the fitness cost function is given by:
$${\rm Cost} = -\log_e({\rm Prob})$$
    * Minimizing the cost function maximizes the posterior
* Full posterior can be approximated by calculating the curvature of the log-probability (or cost-function)
* Covariance matrix is given by the inverse the curvature matrix (Hessian):
$$\Sigma \approx - H^{-1}$$
